# Part 1: Sampling Data
---

## Accessing Data
At this stage I now had my data stored in 'MyDB' on CasJobs. Fortuitously, CasJobs is actually only one part of a platform called [SciServer](https://apps.sciserver.org), which has apps for database access, cloud computing with both interactive notebooks and asynchronous running of either a script or a notebook, and integrated access to an online file hosting service called [SciDrive](https://www.scidrive.org). SciServer actually has a package to access all of these apps from either a notebook on their server, or anywhere else ([API](http://www.sciserver.org/docs/sciscript-python/)). 

As a disclaimer, none of the functions written with these modules will work if run unless altered, as they all require the user to be logged in, and they are all written with assumptions about my setup (i.e. database, table names, locations, etc). *(Well actually they would still work if I hadn't removed my username and password from the respective login functions, but I'm sure you can understand why I did so.)*

In [ ]:
from SciServer import CasJobs, SkyQuery, SciDrive
from imblearn.under_sampling import RandomUnderSampler
import pandas as pd
import sample_process as sp

Defining some base variables for accessing tables. *(Note: SkyQuery is another database access app. Although it can also run SQL queries, it seems to be more set up for viewing and accessing data tables than querying.)*

In [ ]:
#for SkyQuery:
table_source = 'webuser.AllStars'
database = 'MyDB'
#for CasJobs:
table = 'AllStars'
index_query = f'''SELECT specobjid
                FROM {database}.{table}
                '''
row_indices = list(CasJobs.executeQuery(sql=index_query, context=database)['specobjid'])
rows = len(row_indices)

I wanted to subsample my data columns in a variety of ways to test out various features combinations for modelling. 

In [ ]:
paths = {'SomeStars':[], 'SomeStars_no_err':['err'], 'SomeStars_no_side':['side'],
         'SomeStars_no_cont': ['cont'], 'SomeStars_just_cont':['err', 'side'],
         'SomeStars_just_side':['err', 'cont']}

sp.subsample() and sp.make_column_string are both functions from the sample_process module I wrote.  
- subsample(rows, percent, seed=42) returns SQL query pieces to pull a random subset of rows from the database table, using ORDER BY RAND(seed).  
- make_column_string(table_source, dataset, \*args) returns a string with all column names to grab with SQL query, comma separated. Optional args can contain one or more of 'err', 'side', 'cont', or 'mask'. Including any as an argument will drop the corresponding column for each line index.
This cell creates a query to pull a random 5% of the dataset for each path in the above dict, without the given columns.  

I started with only 5% of the dataset because my full dataset contains over 500,000 rows.

In [ ]:
queries = {}
for path, cols in paths.items():
    s = sp.subsample(rows, 0.05)
    queries[path] = f'''SELECT {s[0]} {sp.make_sql_column_string(table_source, dataset, *cols)}
    FROM {database}.{table}
    {s[1]}'''

This cell executes each of the above queries and uploads the result into a csv file in a SciDrive folder.

In [ ]:
for path, query in queries.items():
    try: 
        df = CasJobs.executeQuery(sql=query, context=context)
        SciDrive.upload(path=f'metis_project_3/{path}.csv', data = df.to_csv())
    except:
        sp.login()
        df = CasJobs.executeQuery(sql=query, context=context)
        SciDrive.upload(path=f'metis_project_3/{path}.csv', data = df.to_csv())